In [1]:
import sys
import matplotlib
if 'matplotlib.pyplot' not in sys.modules.keys():
    matplotlib.use("Agg")
import matplotlib.pyplot as plt
import json
import numpy as np

%matplotlib notebook

DeltInx=[100, 300, 200]

class IMUData():
    def __init__(self):

        self.time = []; self.temp = []
        self.ax = []; self.ay = []; self.az = []
        self.mx = []; self.my = []; self.mz = []
        self.gx = []; self.gy = []; self.gz = []
        
        self.timeRpt = []; self.tempRpt = []
        self.axRpt = []; self.ayRpt = []; self.azRpt = []
        self.mxRpt = []; self.myRpt = []; self.mzRpt = []
        self.gxRpt = []; self.gyRpt = []; self.gzRpt = []
        
class SweepData():
    def __init__(self):
        self.time = []
        self.pip0 = []; self.pip1 = [];
        
        self.timeRpt = []
        self.pip0Rpt = []; self.pip1Rpt = [];
        
class PayloadData():
    def __init__(self):
        self.rawData = ''
        self.imu = IMUData()
        self.sweeps = SweepData()
        self.macAddress = ''
        self.rssiData = []
        self.shieldID = 0
        
# Functions for merging bytes
def combine2(a, b):
    return (a << 8) | b
def combine4(a, b, c, d):
    return (a << 24) | (b << 16) | (c << 8) | d

# Functions for saving data
def convertDataToLists(dataSet):
    dataSet.imu.ax = dataSet.imu.ax.tolist()
    dataSet.imu.ay = dataSet.imu.ay.tolist()
    dataSet.imu.az = dataSet.imu.az.tolist()
    dataSet.imu.mx = dataSet.imu.mx.tolist()
    dataSet.imu.my = dataSet.imu.my.tolist()
    dataSet.imu.mz = dataSet.imu.mz.tolist()
    dataSet.imu.gx = dataSet.imu.gx.tolist()
    dataSet.imu.gy = dataSet.imu.gy.tolist()
    dataSet.imu.gz = dataSet.imu.gz.tolist()
    dataSet.imu.temp = dataSet.imu.temp.tolist()
    dataSet.imu.time = dataSet.imu.time.tolist()
    dataSet.sweeps.time = dataSet.sweeps.time.tolist()
    dataSet.sweeps.pip0 = dataSet.sweeps.pip0.tolist()
    dataSet.sweeps.pip1 = dataSet.sweeps.pip1.tolist()
    
    dataSet.imu.axRpt = dataSet.imu.axRpt.tolist()
    dataSet.imu.ayRpt = dataSet.imu.ayRpt.tolist()
    dataSet.imu.azRpt = dataSet.imu.azRpt.tolist()
    dataSet.imu.mxRpt = dataSet.imu.mxRpt.tolist()
    dataSet.imu.myRpt = dataSet.imu.myRpt.tolist()
    dataSet.imu.mzRpt = dataSet.imu.mzRpt.tolist()
    dataSet.imu.gxRpt = dataSet.imu.gxRpt.tolist()
    dataSet.imu.gyRpt = dataSet.imu.gyRpt.tolist()
    dataSet.imu.gzRpt = dataSet.imu.gzRpt.tolist()
    dataSet.imu.tempRpt = dataSet.imu.tempRpt.tolist()
    dataSet.imu.timeRpt = dataSet.imu.timeRpt.tolist()
    dataSet.sweeps.timeRpt = dataSet.sweeps.timeRpt.tolist()
    dataSet.sweeps.pip0Rpt = dataSet.sweeps.pip0Rpt.tolist()
    dataSet.sweeps.pip1Rpt = dataSet.sweeps.pip1Rpt.tolist()
    return dataSet

def makeJSONFile (fname, payloadData):
    f = open(fname, 'w')
    # Convert into lists and dictionaries for JSON storage
    payloadData = convertDataToLists(payloadData)
    JSONDict = {}
    # First the IMU data
    imu = payloadData.imu
    imuData = {"time":imu.time, "temp":imu.temp,
               "ax":imu.ax, "ay":imu.ay, "az":imu.az,
               "mx":imu.mx, "my":imu.my, "mz":imu.mz,
               "gx":imu.gx, "gy":imu.gy, "gz":imu.gz}
    JSONDict["imu"] = imuData
    # Then the sweep data
    sweeps = payloadData.sweeps
    sweepData = {"id":sweeps.payloadID, "time":sweeps.time, "pip0":sweeps.pip0, "pip1":sweeps.pip1}
    JSONDict["sweeps"] = sweepData

    json.dump(JSONDict, f)
    f.close()
    
# Functions for padding data

# padding time data as lists
def timefix(l, new, dt):   # l = time data as list, new = empty list, dt = delta t
    for t in l:
        if l.index(t) >= 2:
            # replace bad timewords
            if t - l[l.index(t) - 1] < 0 or t - l[l.index(t) - 1] > 1:
                l[l.index(t) - 1] = l[l.index(t) - 2] + dt
                t = l[l.index(t) - 1] + dt



In [5]:
path = "./calc_startTime/"
file_str = "data_file_230k-Shield02_startTime_test%s.txt"; numfiles=6
file_str = "data_file_230k-Shield22_startTime_test%s.txt"; numfiles=3
tarr = []
for num in np.arange(1,numfiles+1,1): 
    file_name = file_str % num
    dataFile = path + file_name

    ########### Load the data file ###########
    f = open(dataFile, 'r')
    rawData = f.read()

    # Create payload objects and store parsed data
    mainPIPData = PayloadData()
    mainPIPData.rawData = rawData
    payloads = [mainPIPData]

    ########### Parse by data type for each payload ###########
    strict_parse = False # Require a pound symbol at the end of the data
    for payload in payloads:
        payloadrawData = payload.rawData
        imu = []
        sweeps = []
        imuRpt = []
        sweepsRpt = []
        interrupt = []
        # 2 PIPs with 28 2-byte samples + 4-byte timestamp + ID
        sweepSamples = 28
        numSweepBytes = sweepSamples * 2 * 2 + 4 + 1
        # 9 2-byte data points + 2-byte temp + 4-byte timestamp
        numIMUBytes = 24
        # Barium interrupt
        numInterruptBytes = 4

        pntr = 0
        while pntr < len(payloadrawData):
            # Check message start sentinels and not at end
            if payloadrawData[pntr] == '#' and (len(payloadrawData)-pntr) >= numIMUBytes:
                if payloadrawData[pntr+1] == 'I':
                    # If not the last, check message end sentinel
                    if pntr+2+numIMUBytes < len(payloadrawData):
                        if not strict_parse or payloadrawData[pntr+2+numIMUBytes] == '#':
                            imu.append(payloadrawData[pntr+2:pntr+2+numIMUBytes])
                    # Or just add the last message
                    else:
                        imu.append(payloadrawData[pntr+2:pntr+2+numIMUBytes])
                    pntr += 1
                elif payloadrawData[pntr+1] == 'S':
                    if  pntr+2+numSweepBytes < len(payloadrawData):
                        if not strict_parse or payloadrawData[pntr+2+numSweepBytes] == '#':
                            sweeps.append(payloadrawData[pntr+2:pntr+2+numSweepBytes])
                    else:
                        sweeps.append(payloadrawData[pntr+2:pntr+2+numSweepBytes])
                    pntr += 1
                elif payloadrawData[pntr+1] == 'J':
                    if pntr+2+numIMUBytes < len(payloadrawData):
                        if not strict_parse or payloadrawData[pntr+2+numIMUBytes] == '#':
                            imuRpt.append(payloadrawData[pntr+2:pntr+2+numIMUBytes])
                    else:
                        imuRpt.append(payloadrawData[pntr+2:pntr+2+numIMUBytes])
                    pntr += 1
                elif payloadrawData[pntr+1] == 'T':
                    if  pntr+2+numSweepBytes < len(payloadrawData):
                        if not strict_parse or payloadrawData[pntr+2+numSweepBytes] == '#':
                            sweepsRpt.append(payloadrawData[pntr+2:pntr+2+numSweepBytes])
                    else:
                        sweepsRpt.append(payloadrawData[pntr+2:pntr+2+numSweepBytes])
                    pntr += 1
                elif payloadrawData[pntr+1] == 'B':
                    if  pntr+2+numInterruptBytes < len(payloadrawData):
                        if not strict_parse or payloadrawData[pntr+2+numInterruptBytes] == '#':
                            interrupt.append(payloadrawData[pntr+2:pntr+2+numInterruptBytes])
                    else:
                        interrupt.append(payloadrawData[pntr+2:pntr+2+numInterruptBytes])
                    pntr += 1
                else:
                    pntr += 1
            else:
                pntr += 1
        print("Num IMU Messages: %s" %len(imu))
        print("Num Sweep Messages: %s" %len(sweeps))
        print("Num IMU Messages (buffer): %s" %len(imuRpt))
        print("Num Sweep Messages (buffer): %s" %len(sweepsRpt))
        print("Num interrupt messages: %s" %len(interrupt))

        ########### Build data structures ###########
        # First the IMU/IMU Buffer
        imuTime = [];
        ax = []; ay = []; az = []
        mx = []; my = []; mz = []
        gx = []; gy = []; gz = []
        temp = [];

        imuTimeRpt = [];
        axRpt = []; ayRpt = []; azRpt = []
        mxRpt = []; myRpt = []; mzRpt = []
        gxRpt = []; gyRpt = []; gzRpt = []
        tempRpt = [];

        for i in imu:
            if len(i) == numIMUBytes:
                imuTime.append(combine4(ord(i[3]), ord(i[2]), ord(i[1]), ord(i[0])))
                # The byte order is reversed, but the data sets are not
                ax.append(combine2(ord(i[5]), ord(i[4])))
                ay.append(combine2(ord(i[7]), ord(i[6])))
                az.append(combine2(ord(i[9]), ord(i[8])))
                mx.append(combine2(ord(i[11]), ord(i[10])))
                my.append(combine2(ord(i[13]), ord(i[12])))
                mz.append(combine2(ord(i[15]), ord(i[14])))
                gx.append(combine2(ord(i[17]), ord(i[16])))
                gy.append(combine2(ord(i[19]), ord(i[18])))
                gz.append(combine2(ord(i[21]), ord(i[20])))
                temp.append(combine2(ord(i[23]), ord(i[22])))

        for i in imuRpt:
            if len(i) == numIMUBytes:
                imuTimeRpt.append(combine4(ord(i[3]), ord(i[2]), ord(i[1]), ord(i[0])))
                axRpt.append(combine2(ord(i[5]), ord(i[4])))
                ayRpt.append(combine2(ord(i[7]), ord(i[6])))
                azRpt.append(combine2(ord(i[9]), ord(i[8])))
                mxRpt.append(combine2(ord(i[11]), ord(i[10])))
                myRpt.append(combine2(ord(i[13]), ord(i[12])))
                mzRpt.append(combine2(ord(i[15]), ord(i[14])))
                gxRpt.append(combine2(ord(i[17]), ord(i[16])))
                gyRpt.append(combine2(ord(i[19]), ord(i[18])))
                gzRpt.append(combine2(ord(i[21]), ord(i[20])))
                tempRpt.append(combine2(ord(i[23]), ord(i[22])))

        imuTime = np.array(imuTime, dtype='uint32')
        ax = np.array(ax, dtype='int16')
        ay = np.array(ay, dtype='int16')
        az = np.array(az, dtype='int16')
        mx = np.array(mx, dtype='int16')
        my = np.array(my, dtype='int16')
        mz = np.array(mz, dtype='int16')
        gx = np.array(gx, dtype='int16')
        gy = np.array(gy, dtype='int16')
        gz = np.array(gz, dtype='int16')
        temp = np.array(temp, dtype='int16')

        imuTimeRpt = np.array(imuTimeRpt, dtype='uint32')
        axRpt = np.array(axRpt, dtype='int16')
        ayRpt = np.array(ayRpt, dtype='int16')
        azRpt = np.array(azRpt, dtype='int16')
        mxRpt = np.array(mxRpt, dtype='int16')
        myRpt = np.array(myRpt, dtype='int16')
        mzRpt = np.array(mzRpt, dtype='int16')
        gxRpt = np.array(gxRpt, dtype='int16')
        gyRpt = np.array(gyRpt, dtype='int16')
        gzRpt = np.array(gzRpt, dtype='int16')
        tempRpt = np.array(tempRpt, dtype='int16')

        payload.imu.time = imuTime
        payload.imu.ax = ax
        payload.imu.ay = ay
        payload.imu.az = az
        payload.imu.mx = mx
        payload.imu.my = my
        payload.imu.mz = mz
        payload.imu.gx = gx
        payload.imu.gy = gy
        payload.imu.gz = gz
        payload.imu.temp = temp

        payload.imu.timeRpt = imuTimeRpt
        payload.imu.axRpt = axRpt
        payload.imu.ayRpt = ayRpt
        payload.imu.azRpt = azRpt
        payload.imu.mxRpt = mxRpt
        payload.imu.myRpt = myRpt
        payload.imu.mzRpt = mzRpt
        payload.imu.gxRpt = gxRpt
        payload.imu.gyRpt = gyRpt
        payload.imu.gzRpt = gzRpt
        payload.imu.tempRpt = tempRpt

        # Then the sweeps
        payloadID = [];
        sweepTime = []
        allSweepsPIP0 = []
        individualSweepsPIP0 = []
        allSweepsPIP1 = []
        individualSweepsPIP1 = []
        PIP0StartByte = 4 + 1                             # Four for time, one payload ID  
        PIP0StopByte = PIP0StartByte + sweepSamples*2   # Number of samples, two bytes each
        PIP1StopByte = PIP0StopByte + sweepSamples*2      # Number of samples, two bytes each

        sweepTimeRpt = []
        allSweepsPIP0Rpt = []
        individualSweepsPIP0Rpt = []
        allSweepsPIP1Rpt = []
        individualSweepsPIP1Rpt = []
        PIP0StartByteRpt = 4 + 1                             # Four for time, one payload ID  
        PIP0StopByteRpt = PIP0StartByteRpt + sweepSamples*2   # Number of samples, two bytes each
        PIP1StopByteRpt = PIP0StopByteRpt + sweepSamples*2      # Number of samples, two bytes each

        for i in sweeps:
            if len(i) == numSweepBytes:
                sweepTime.append(combine4(ord(i[3]), ord(i[2]), ord(i[1]), ord(i[0])))
                payloadID.append(ord(i[4]))
                tempSweep = []
                for byte in xrange(PIP0StartByte,PIP0StopByte,2):
                    allSweepsPIP0.append(combine2(ord(i[byte+1]), ord(i[byte])))
                    tempSweep.append(combine2(ord(i[byte+1]), ord(i[byte])))
                individualSweepsPIP0.append(tempSweep)
                tempSweep = []
                for byte in xrange(PIP0StopByte,PIP1StopByte,2):
                    allSweepsPIP1.append(combine2(ord(i[byte+1]), ord(i[byte])))
                    tempSweep.append(combine2(ord(i[byte+1]), ord(i[byte])))
                individualSweepsPIP1.append(tempSweep)

        for i in sweepsRpt:
            if len(i) == numSweepBytes:
                sweepTimeRpt.append(combine4(ord(i[3]), ord(i[2]), ord(i[1]), ord(i[0])))
                tempSweepRpt = []
                for byte in xrange(PIP0StartByteRpt,PIP0StopByteRpt,2):
                    allSweepsPIP0Rpt.append(combine2(ord(i[byte+1]), ord(i[byte])))
                    tempSweepRpt.append(combine2(ord(i[byte+1]), ord(i[byte])))
                individualSweepsPIP0Rpt.append(tempSweepRpt)
                tempSweepRpt = []
                for byte in xrange(PIP0StopByteRpt,PIP1StopByteRpt,2):
                    allSweepsPIP1Rpt.append(combine2(ord(i[byte+1]), ord(i[byte])))
                    tempSweepRpt.append(combine2(ord(i[byte+1]), ord(i[byte])))
                individualSweepsPIP1Rpt.append(tempSweepRpt)

        interruptTime = []        
        for i in interrupt:
            if len(i) == numInterruptBytes:
                interruptTime.append(combine4(ord(i[3]), ord(i[2]), ord(i[1]), ord(i[0])))

        payloadMatch = (len(set(payloadID)) == 1)
        if payloadMatch:
            print("Payload ID's match. This was payload #%s." %payloadID[0])
        else:
            print("Payload ID's don't match. Something went wrong...")

        sweepTime = np.array(sweepTime, dtype='uint32')
        individualSweepsPIP0 = np.array(individualSweepsPIP0, dtype='int16')
        individualSweepsPIP1 = np.array(individualSweepsPIP1, dtype='int16')
        payload.shieldID = payloadID[0]
        payload.sweeps.time = sweepTime
        payload.sweeps.pip0 = individualSweepsPIP0
        payload.sweeps.pip1 = individualSweepsPIP1

        sweepTimeRpt = np.array(sweepTimeRpt, dtype='uint32')
        individualSweepsPIP0Rpt = np.array(individualSweepsPIP0Rpt, dtype='int16')
        individualSweepsPIP1Rpt = np.array(individualSweepsPIP1Rpt, dtype='int16')
        payload.sweeps.timeRpt = sweepTimeRpt
        payload.sweeps.pip0Rpt = individualSweepsPIP0Rpt
        payload.sweeps.pip1Rpt = individualSweepsPIP1Rpt
    
    tmp = []
    prev=np.unique(np.array(interruptTime)/10.**6)[0]
    for event in np.unique(np.array(interruptTime)/10.**6): 
        if int(np.floor(event-prev)) >= 30: print "-----------------------------------------"
        print "Number of recordings for %ss:" % (event), len(np.where(np.array(interruptTime)/10.**6 == event)[0]) 
        prev=event
        tmp.append(event)
    tarr.append(tmp)
    del tmp 

Num IMU Messages: 2394
Num Sweep Messages: 2379
Num IMU Messages (buffer): 1948
Num Sweep Messages (buffer): 1948
Num interrupt messages: 500
Payload ID's match. This was payload #22.
Number of recordings for 8.00622s: 500
Num IMU Messages: 2434
Num Sweep Messages: 2409
Num IMU Messages (buffer): 1990
Num Sweep Messages (buffer): 1990
Num interrupt messages: 500
Payload ID's match. This was payload #22.
Number of recordings for 8.00622s: 500
Num IMU Messages: 2431
Num Sweep Messages: 2402
Num IMU Messages (buffer): 1986
Num Sweep Messages (buffer): 1986
Num interrupt messages: 500
Payload ID's match. This was payload #22.
Number of recordings for 8.00622s: 500


In [3]:
# tarr

In [4]:
tarr02 = [val[0] for val in tarr]
print 'Start Times for Shield 2:', tarr02

Start Times for Shield 2: [8.006224, 8.006224, 8.006224, 8.006224, 8.006228, 8.00622]


In [6]:
tarr22 = [val[0] for val in tarr]
print 'Start Times for Shield 22:', tarr22

Start Times for Shield 22: [8.00622, 8.00622, 8.00622]


In [7]:
print 'Start Times (seconds) for Shield 2:', tarr02
print 'Start Times (seconds) for Shield 22:', tarr22

Start Times (seconds) for Shield 2: [8.006224, 8.006224, 8.006224, 8.006224, 8.006228, 8.00622]
Start Times (seconds) for Shield 22: [8.00622, 8.00622, 8.00622]


In [8]:
# del tarr
tarr=[]
tarr.extend(tarr02); tarr.extend(tarr22)
print 'Start Times (seconds):', tarr

Start Times (seconds): [8.006224, 8.006224, 8.006224, 8.006224, 8.006228, 8.00622, 8.00622, 8.00622, 8.00622]


In [11]:
print np.round(np.mean(tarr), decimals=6)
tarr.extend(tarr22)
print np.round(np.mean(tarr), decimals=6)

8.006223
8.006222


In [12]:
import numpy as np
np.round(1493.543352-8.00622, decimals=6)
EVT=1493.543352

In [16]:
print 'Interrupt Time with dt=8s:       ', np.round(EVT-8, decimals=6), 's'

print 'Interrupt Time with dt=8.00622s: ', np.round(EVT-8.00622, decimals=6), 's'
print 'Interrupt Time with dt=8.006222s:', np.round(EVT-8.006222, decimals=6), 's'
print 'Interrupt Time with dt=8.006223s:', np.round(EVT-8.006223, decimals=6), 's'



Interrupt Time with dt=8s:        1485.543352 s
Interrupt Time with dt=8.00622s:  1485.537132 s
Interrupt Time with dt=8.006222s: 1485.53713 s
Interrupt Time with dt=8.006223s: 1485.537129 s


In [ ]:
1485.537152